In [ ]:
# Import Libraries
import warnings
warnings.filterwarnings("ignore")

import os
import glob
import matplotlib.pyplot as plt

# Import Keras
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.preprocessing.image import ImageDataGenerator

# Set the path to the training and testing data directories
train_dir = "/content/drive/My Drive/DATA WAREHOUSE/Ready/Apple/TRAIN"
test_dir = "/content/drive/My Drive/DATA WAREHOUSE/Ready/Apple/TEST"

# Function to get the count of images in a directory
def get_files(directory):
    if not os.path.exists(directory):
        return 0
    count = 0
    for current_path, dirs, files in os.walk(directory):
        for dr in dirs:
            count += len(glob.glob(os.path.join(current_path, dr + "/*")))
    return count

# Get the number of classes, training samples, and test samples
train_samples = get_files(train_dir)
num_classes = len(glob.glob(train_dir + "/*"))
test_samples = get_files(test_dir)

print(num_classes, "Classes")
print(train_samples, "Train images")
print(test_samples, "Test images")

# Preprocessing data.
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    validation_split=0.2,  # Validation split 20%.
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)

# Set image dimensions and batch size
img_width, img_height = 256, 256
input_shape = (img_width, img_height, 3)
batch_size = 32

# Create data generators for training and testing
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    shuffle=True,
    target_size=(img_width, img_height),
    batch_size=batch_size
)

# Define class labels
class_labels = train_generator.class_indices

# Build CNN Architecture
model = Sequential()
model.add(Conv2D(32, (5, 5), input_shape=input_shape, activation='relu'))
model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

# Compile the model
opt = keras.optimizers.Adam(lr=0.001)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

# Model training
epochs = 10
history = model.fit_generator(
    train_generator,
    epochs=epochs,
    steps_per_epoch=train_samples // batch_size,
    validation_data=train_generator,
    validation_steps=train_samples // batch_size,
    verbose=1
)

# Plot training metrics
plt.figure(figsize=(12, 4))

# Training and validation accuracy
plt.subplot(1, 2, 1)
plt.plot(range(1, epochs + 1), history.history['acc'], 'b', label='Training Accuracy')
plt.plot(range(1, epochs + 1), history.history['val_acc'], 'r', label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

# Training and validation loss
plt.subplot(1, 2, 2)
plt.plot(range(1, epochs + 1), history.history['loss'], 'b', label='Training Loss')
plt.plot(range(1, epochs + 1), history.history['val_loss'], 'r', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

# Evaluate the model on the test data
score, accuracy = model.evaluate(test_generator, verbose=1)
print("Test score is {}".format(score))
print("Test accuracy is {}".format(accuracy))
